# GMAC Acceleration Magnitude: Grid search of GMAC parameters

The acceleration magnitude estimation for the GMAC algorithm has the following parameters:
1. Butteworth highpass filter cut-off for accl. mag. estimation. $\left( f_{c} \right)$
2. Butteworth highpass filter order for accl. mag. estimation. $\left( n_{c} \right)$
3. Acceleration deadband threshold. $\left( a_{th0} \right)$
4. Moving average window for acceleration magnitude estimation. $\left( n_{am} \right)$

### Getting the data

If you do not already have the data to run this this notebook, you need to
download it from here.



### Standards modules

In [1]:
%reload_ext autoreload
%autoreload 2

In [46]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
import scipy
from scipy import signal
import pathlib
import itertools
import glob
import json

In [3]:
import seaborn as sns

In [4]:
# Define a dictionary containing the font settings for LaTeX
latex_font = {
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{amsmath}",  # Optional: Include additional LaTeX packages
    "font.family": "serif",  # Use a serif font (e.g., Times New Roman)
    "font.serif": ["Times New Roman"],  # Specify the font family
}

# Update the Matplotlib configuration with the LaTeX font settings
plt.rcParams.update(latex_font)

### Custom Modules

In [8]:
sys.path.append("../scripts")

import classification_algorithms as ca
import reduced_models as rm
import task_analysis as ta

import misc

In [9]:
import gmac

In [10]:
# Define the font family and size
# font = {'family': 'sans-serif', 'sans-serif': ['Helvetica']}

# # Set the font properties for various elements
# plt.rc('font', **font)

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.weight'] = 'light'  # Use 'light' font weight


## Define notebook level constants

In [11]:
# Sampling rate for the data is 50ms
dT = 0.02
Fs = int(1 / dT)

## Read the Controls and Patients Data

In [13]:
# Read healthy and control data
left, right = misc.read_data(subject_type='control')
aff, unaff = misc.read_data(subject_type='patient')

# Assign segments for each subject
left = pd.concat([misc.assign_segments(left[left.subject == subj], dur_th=1, dT=dT)
                  for subj in left.subject.unique()], axis=0)
right = pd.concat([misc.assign_segments(right[right.subject == subj], dur_th=1, dT=dT)
                   for subj in right.subject.unique()])
aff = pd.concat([misc.assign_segments(aff[aff.subject == subj], dur_th=1, dT=dT)
                 for subj in aff.subject.unique()])
unaff = pd.concat([misc.assign_segments(unaff[unaff.subject == subj], dur_th=1, dT=dT)
                   for subj in unaff.subject.unique()])

# All limbs data ddf
datadf = {
    "left": left,
    "right": right,
    "aff": aff,
    "unaff": unaff
}

#### Parameter ranges for the grid search

In [47]:
basedir = "../data/output"

# Make directory if it does not exist
outdir = pathlib.Path(basedir, "am")
outdir.mkdir(parents=True, exist_ok=True)

# GMAC Parameter ranges
am_param_ranges = {
    "fc": [0.01, 0.1, 1],
    "nc": [1, 2, 4],
    "nam": list(map(int, [Fs, 5*Fs, 10*Fs]))
}

# Save params file.
with open(pathlib.Path(outdir, "am_params.json"), "w") as f:
    json.dump(am_param_ranges, f, indent=4)

dfcols = ["subject"] + list(am_param_ranges.keys()) + ["am", "vm"]

In [19]:
for limbkey in ["left", "right", "aff", "unaff"]:
    # Which limb?
    limbdf = datadf[limbkey]
    subjects = limbdf.subject.unique()

    # Number of all possible combinations
    Ncombs = len(list(misc.generate_param_combinations_am(am_param_ranges)))

    # All parameter combinations.
    param_combs = list(misc.generate_param_combinations_am(am_param_ranges))

    # Estimate pitch for each subject.
    accl_cols = ['ax', 'ay', 'az']
    dfcols = ["subject"] + list(am_param_ranges.keys()) + ["am", "vm"]

    for i, subj in enumerate(subjects):
        am_df = pd.DataFrame(columns=dfcols)
        # Go through all parameter combinations.
        _sinx = limbdf.subject == subj
        for j, _pc in enumerate(param_combs):
            sys.stdout.write(f"\r{limbkey} {i:3d}, {j:3d}")
            df = limbdf[limbdf.subject==subj]
            # Compute the vector magnitude
            _vmdf = misc.compute_vector_magnitude(df)
            # Compute the accl magnitude
            _tempam = gmac.estimate_accl_mag(accl=df[accl_cols].values,
                                            fs=Fs, fc=_pc["fc"], nc=_pc["nc"],
                                            n_am=_pc["nam"])
            _tempamdf = pd.DataFrame(np.vstack((_tempam, df.segment.values)).T,
                                    columns=["am", "segment"], index=df.index)
            # Resample accl magnitude to the same frequency as vector magnitude
            _amdf = pd.concat([_y.resample(str(1) + 'S').sum()
                            for _, _y in _tempamdf.groupby('segment')])
            
            # Merge the two dataframes
            _tempdf = pd.merge(_vmdf, _amdf["am"], left_index=True, right_index=True)

            # Update amdf
            _n = len(_tempdf)
            _subj_am = np.hstack((subj * np.ones((_n, 1)),
                                _pc["fc"] * np.ones((_n, 1)),
                                _pc["nc"] * np.ones((_n, 1)),
                                _pc["nam"] * np.ones((_n, 1)),
                                _tempdf.am.values.reshape(-1, 1),
                                _tempdf.counts.values.reshape(-1, 1)))
            am_df = pd.concat((am_df, pd.DataFrame(_subj_am, columns=dfcols)),
                            ignore_index=True)
        # Save data regularly
        am_df.to_csv(pathlib.Path(outdir, f"raw_am_{limbkey}_{subj}.csv").as_posix(), index=False)


unaff   4,  26